In [189]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [ ]:
n_customers = 100
n_products = 10
n_days = 180
product_names = ['Avocado','Banana','Beef','Cauliflower','Egg','Milk','Shrimp','Tomato','Yogurt','Zucchini']

#### **Part I** demand simulation

#### generate temp and CPI data

In [209]:
def gen_macro_df(days, cpi_low, cpi_high, temp_low, temp_high, mcpi_low, mcpi_high):    
    # Step 1: Generate dates spanning 6 months to today
    end_date = datetime.today()
    start_date = end_date - timedelta(days=days)  # Approximate 6 months as 180 days
    dates = pd.date_range(start=start_date, end=end_date, freq='D')
    
    # Step 2: Generate temperature values with increasing trend and random noise
    np.random.seed(0)  # For reproducibility
    num_days = len(dates)
    temperature_base = np.linspace(temp_low, temp_high, num_days)  # Base temperature increasing linearly from 20 to 30
    temperature_noise = np.random.normal(0, 5, num_days)  # Adding random noise with mean 0 and std deviation 2
    temperatures = np.round(temperature_base + temperature_noise, 2)
    
    rainfalls = np.round(np.random.normal(0, 100, num_days), 2) 
    rainfalls = np.where(rainfalls<0, 0, rainfalls)

    # Step 3: Generate CPI values that vary each month with random noise
    months = pd.date_range(start=start_date, end=end_date, freq='M')
    cpi_base = np.linspace(cpi_low, cpi_high, len(months))  # Base CPI increasing linearly from 100 to 105
    cpi_noise = np.random.normal(0, 1, len(months))  # Adding random noise with mean 0 and std deviation 1
    cpi_values = np.round(cpi_base + cpi_noise, 2)

    # Step 4: Generate CMPI values that vary each month with random noise
    mcpi_base = np.linspace(mcpi_low, mcpi_high, len(months))  # Base MCPI increasing linearly from 100 to 105
    mcpi_noise = np.random.normal(0, 1, len(months))  # Adding random noise with mean 0 and std deviation 1
    mcpi_values = np.round(mcpi_base + mcpi_noise, 2)
    
    # Create a DataFrame with dates and temperatures
    df = pd.DataFrame({'Date': dates, 'Temperature': temperatures, 'Rainfall': rainfalls})
    
    # Assign CPI values to the first day of each month, and forward fill the rest of the days in the month
    cpi_series = pd.Series(cpi_values, index=months)
    mcpi_series = pd.Series(mcpi_values, index=months)
    
    # Ensure the CPI series starts from the first date in the DataFrame
    cpi_series = cpi_series.reindex(dates, method='ffill').fillna(method='bfill')
    mcpi_series = mcpi_series.reindex(dates, method='ffill').fillna(method='bfill')
    
    # Assign the CPI values to the DataFrame
    df['CPI'] = cpi_series.values
    df['MCPI'] = cpi_series.values
    
    # Fix the date format to 'YYYY-MM-DD'
    df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

    return df
    
df_macro = gen_macro_df(days=180, cpi_low=100, cpi_high=105, temp_low=40, temp_high=80, mcpi_low=50, mcpi_high=250)

#### generate customer data

In [191]:
def gen_cust_df(n_customers, income_mean, income_std):    
    # Number of customers
    n_customers = n_customers
    
    # Generate customer IDs
    customer_ids = np.arange(1, n_customers + 1)
    
    # Generate random values for female
    np.random.seed(0)  # For reproducibility
    female = np.random.choice([0, 1], size=n_customers)
    
    # Generate random values for age (between 18 and 70)
    age = np.random.randint(18, 71, size=n_customers)
    
    # Generate random values for income (normal distribution with mean 50k and std deviation 15k)
    income = np.random.normal(income_mean, income_std, size=n_customers)
    income = np.round(income, 2)  # Round to 2 decimal places
    
    # Create a DataFrame with the generated data
    data = pd.DataFrame({
        'CustomerID': customer_ids,
        'Female': female,
        'Age': age,
        'Income': income
    })
    
    return data

df_customer = gen_cust_df(n_customers=100, income_mean=50000, income_std=30000)

#### generate order data

In [210]:
# Create order quantities for date, customer, and product combination
order_quantities = np.random.randint(0, 10, size=(n_customers, n_products, n_days))

# Create a meshgrid for customer IDs, product IDs, and days
customers = np.arange(n_customers) + 1
products = np.arange(n_products) + 1
# days = np.arange(n_days)
end_date = datetime.today()
start_date = end_date - timedelta(days=n_days-1)  # Approximate 6 months as 180 days
dates = pd.date_range(start=start_date, end=end_date, freq='D')

customers_grid, products_grid, dates_grid = np.meshgrid(customers, products, dates, indexing='ij')

df_orders = pd.DataFrame({
    'CustomerID': customers_grid.flatten(),
    'ProductID': products_grid.flatten(),
    'Date': dates_grid.flatten(),
    'OrderQuantity': order_quantities.flatten()
})

df_orders['OrderQuantity'] = np.where(df_orders['Date']==df_orders['Date'].min(), df_orders['OrderQuantity'], 0)
df_orders.sort_values(by=['CustomerID', 'ProductID', 'Date'], inplace=True)
df_orders['OrderQuantity_lag1'] = df_orders.groupby(['CustomerID', 'ProductID'])['OrderQuantity'].shift(1)
df_orders['Date'] = df_orders['Date'].dt.strftime('%Y-%m-%d')
df_orders['Day'] = df_orders['Date'].rank(method='dense').astype(int)

product_map = {i+1: name for i, name in enumerate(product_names)}
df_orders['Product'] = df_orders['ProductID'].map(product_map)

df_orders.head(2)

,CustomerID,ProductID,Date,OrderQuantity,OrderQuantity_lag1,Day,Product
0,1,1,2023-12-22,4,NaN,1,Avocado
1,1,1,2023-12-23,0,4.0,2,Avocado


In [211]:
df = df_orders.merge(df_macro, on='Date', how='left')
df = df.merge(df_customer, on='CustomerID', how='left')
df['intercept']=1
df.head(2)

,CustomerID,ProductID,Date,OrderQuantity,OrderQuantity_lag1,Day,Product,Temperature,Rainfall,CPI,MCPI,Female,Age,Income,intercept
0,1,1,2023-12-22,4,NaN,1,Avocado,42.22,0.0,100.93,100.93,0,60,59662.22,1
1,1,1,2023-12-23,0,4.0,2,Avocado,45.34,0.0,100.93,100.93,0,60,59662.22,1


In [212]:
# define the coefficient for ground truth
data = {
    # 'intercept': [0.017925, -0.928484, -0.881833, -0.747056],
    # 'OrderQuantity_lag1': [0.792873, 0.732268, 0.71704, 0.708182],
    # 'Female': [0.324909, 0.281898, 0.416095, 0.561980],
    # 'Age': [0.12468, 0.184523, 0.161964, 0.108463],
    # 'Income': [0.000046, 0.000025, 0.000036, 0.000026],
    # 'Temperature': [-0.067693, -0.115606, -0.033221, 0.031307],
    # 'Rainfall': [0.013351, -0.002803, 0.006147, 0.016702],
    # 'CPI': [0.004972, 0.002614, -0.002624, -0.003080]
    'intercept': np.random.normal(-0.88, 0.2, n_products),
    'OrderQuantity_lag1': np.random.normal(0.7, 0.1, n_products),
    'Female': np.random.normal(0.3, 0.1, n_products),
    'Age': np.random.normal(0.15, 0.05, n_products),
    'Income': np.random.normal(0.00035, 0.1, n_products),
    'Temperature': np.random.normal(0.1, 0.2, n_products),
    'Rainfall': np.random.normal(-0.2, 0.2, n_products),
    'CPI': np.random.normal(0.005, 0.1, n_products),
    'MCPI': np.random.normal(0.05, 0.01, n_products)
}

# Index for the DataFrame
index = pd.Index([i+1 for i in range(n_products)], name='ProductID')

# Create the DataFrame
coefficients = pd.DataFrame(data, index=index)
coefficients


,intercept,OrderQuantity_lag1,Female,Age,Income,Temperature,Rainfall,CPI,MCPI
ProductID,,,,,,,,,
1,-0.735354,0.612311,0.368606,0.119620,0.106383,0.439065,-0.188795,-0.097291,0.044593
2,-0.966949,0.764884,0.238397,0.150442,0.035677,0.112805,0.022737,0.088773,0.049461
3,-0.798681,0.739667,0.157253,0.204555,-0.104666,0.211887,-0.490446,0.132956,0.045080
4,-0.802164,0.751897,0.255741,0.198306,-0.000354,0.227153,-0.129148,-0.158838,0.022546
5,-0.961655,0.693448,0.229455,0.168273,-0.076526,-0.042742,-0.106288,0.104325,0.053378
6,-1.210599,0.554504,0.280883,0.087296,-0.042017,0.096724,-0.419420,0.056502,0.037403
7,-0.742415,0.695295,0.230845,0.114380,0.073290,0.083268,0.152171,0.129608,0.052046
8,-0.762278,0.783880,0.394680,0.156606,-0.044238,0.249622,-0.305316,0.100548,0.056429
9,-0.512677,0.729260,0.380475,0.163902,-0.037013,0.265098,-0.127567,0.225494,0.051102


In [213]:
def gen_next_product_quantity(df, n_days, prod, coefficients):
    for day in range(2, n_days+1):
        rows_to_mul = df['Day'] == day
        next_product = df.loc[rows_to_mul,coefficients.columns].dot(coefficients[coefficients.index==prod].values[0]).values[0]\
        + np.random.normal(0,4)
        if next_product>=0:
            df.loc[rows_to_mul, 'OrderQuantity'] = int(next_product)
        else:
            df.loc[rows_to_mul, 'OrderQuantity'] = 0
        rows_to_update = df['Day'] == day+1
        df.loc[rows_to_update, 'OrderQuantity_lag1']=df.loc[rows_to_mul, 'OrderQuantity'].values[0]
    return df

products = df['ProductID'].unique()
custs = df['CustomerID'].unique()
df_res = []
for cust in custs:
    for prod in products:
        df_tmp = df[(df['CustomerID']==cust)&(df['ProductID']==prod)]
        df_tmp = gen_next_product_quantity(df=df_tmp, n_days=180, prod=prod, coefficients=coefficients)
        df_res.append(df_tmp)
stage1_data = pd.concat(df_res)
stage1_data.head()


,CustomerID,ProductID,Date,OrderQuantity,OrderQuantity_lag1,Day,Product,Temperature,Rainfall,CPI,MCPI,Female,Age,Income,intercept
0,1,1,2023-12-22,4,NaN,1,Avocado,42.22,0.00,100.93,100.93,0,60,59662.22,1
1,1,1,2023-12-23,6371,4.0,2,Avocado,45.34,0.00,100.93,100.93,0,60,59662.22,1
2,1,1,2023-12-24,10253,6371.0,3,Avocado,51.87,62.52,100.93,100.93,0,60,59662.22,1
3,1,1,2023-12-25,12650,10253.0,4,Avocado,50.23,0.00,100.93,100.93,0,60,59662.22,1
4,1,1,2023-12-26,14112,12650.0,5,Avocado,36.22,0.00,100.93,100.93,0,60,59662.22,1


In [200]:
stage1_data.groupby(['ProductID'])['OrderQuantity'].describe()

,count,mean,std,min,25%,50%,75%,max
ProductID,,,,,,,,
1,18000.0,623.121556,335.117331,0.0,401.00,619.5,805.00,1450.0
2,18000.0,339.785444,1457.078701,0.0,0.00,0.0,0.00,9522.0
3,18000.0,33134.790333,19242.049641,0.0,19701.00,32461.0,43436.25,79853.0
4,18000.0,13975.821444,8090.078745,0.0,8295.00,13657.5,18289.25,33594.0
5,18000.0,31735.736667,18360.451236,0.0,18902.00,31071.5,41625.25,76273.0
6,18000.0,21122.314389,12343.698019,0.0,12585.00,20820.0,27492.00,51315.0
7,18000.0,13697.955889,7893.898610,0.0,8123.75,13392.5,17895.25,32766.0
8,18000.0,4694.597556,2736.306038,0.0,2797.00,4615.5,6123.00,11395.0
9,18000.0,3797.044444,2197.706806,0.0,2257.00,3731.0,4950.25,9117.0


In [214]:
stage1_data.to_csv('./data/stage1_data.csv', index=False)

In [215]:
np.random.seed(0)
stage1_data_store = stage1_data.groupby(['Date', 'ProductID', 'Product']).agg({'OrderQuantity': 'sum',
                                                                    'Temperature': 'mean',
                                                                    'Rainfall': 'mean',
                                                                    'CPI': 'mean',
                                                                    'MCPI': 'mean',
                                                                    'Female': 'mean',
                                                                    'Age': 'mean',
                                                                    'Income': 'mean'
}).reset_index()
stage1_data_store['Inventory'] = stage1_data_store['OrderQuantity'] + np.random.normal(0, 100, n_days*n_products).astype(int)
stage1_data_store['Low_Stock_Risk'] = 1 - (stage1_data_store['OrderQuantity'] / stage1_data_store['Inventory'])

In [217]:
stage1_data_store.sort_values(by=['Date', 'Low_Stock_Risk'], inplace=True)
stage1_data_store['Low_Stock_Priority'] = pd.qcut(stage1_data_store['Low_Stock_Risk'], 10, labels=[f'{i}' for i in range(1, 11)])

stage1_data_store = stage1_data_store[['Date', 'ProductID', 'Product', 'Low_Stock_Priority', 'Low_Stock_Risk', 'OrderQuantity', 'Inventory', 'Temperature', 'Rainfall', 'CPI', 'MCPI',
       'Female', 'Age', 'Income']]
stage1_data_store.to_csv('./data/stage1_data_store.csv', index=False)

In [218]:
stage1_data_store

,Date,ProductID,Product,Low_Stock_Priority,Low_Stock_Risk,OrderQuantity,Inventory,Temperature,Rainfall,CPI,MCPI,Female,Age,Income
5,2023-12-22,6,Milk,1,-0.302181,418,321,42.22,0.00,100.93,100.93,0.56,44.08,49946.1506
7,2023-12-22,8,Tomato,1,-0.031780,487,472,42.22,0.00,100.93,100.93,0.56,44.08,49946.1506
8,2023-12-22,9,Yogurt,1,-0.021053,485,475,42.22,0.00,100.93,100.93,0.56,44.08,49946.1506
1,2023-12-22,2,Banana,10,0.078278,471,511,42.22,0.00,100.93,100.93,0.56,44.08,49946.1506
9,2023-12-22,10,Zucchini,10,0.085062,441,482,42.22,0.00,100.93,100.93,0.56,44.08,49946.1506
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1793,2024-06-18,4,Cauliflower,5,0.000000,535,535,78.23,0.38,104.61,104.61,0.56,44.08,49946.1506
1796,2024-06-18,7,Shrimp,6,0.000002,1232285,1232287,78.23,0.38,104.61,104.61,0.56,44.08,49946.1506
1799,2024-06-18,10,Zucchini,6,0.000049,555052,555079,78.23,0.38,104.61,104.61,0.56,44.08,49946.1506
1791,2024-06-18,2,Banana,7,0.000077,782595,782655,78.23,0.38,104.61,104.61,0.56,44.08,49946.1506
